In [1]:
import string
import collections
import multiprocessing
import itertools
from enigma import *

In [2]:
Signal = collections.namedtuple('Signal', ['bank', 'wire'])
Connection = collections.namedtuple('Connection', ['banks', 'scrambler'])
MenuItem = collections.namedtuple('MenuIem', ['before', 'after', 'number'])

In [3]:
class Scrambler(object):
    def __init__(self, wheel1_spec, wheel2_spec, wheel3_spec, reflector_spec,
                 wheel1_pos='a', wheel2_pos='a', wheel3_pos='a'):
        self.wheel1 = SimpleWheel(wheel1_spec, position=wheel1_pos)
        self.wheel2 = SimpleWheel(wheel2_spec, position=wheel2_pos)
        self.wheel3 = SimpleWheel(wheel3_spec, position=wheel3_pos)
        self.reflector = Reflector(reflector_spec)
    
    def __getattribute__(self, name):
        if name=='wheel_positions':
            return self.wheel1.position, self.wheel2.position, self.wheel3.position 
        elif name=='wheel_positions_l':
            return self.wheel1.position_l, self.wheel2.position_l, self.wheel3.position_l 
        else:
            return object.__getattribute__(self, name)
    
    def advance(self, wheel1=False, wheel2=False, wheel3=True):
        if wheel1: self.wheel1.advance()
        if wheel2: self.wheel2.advance()
        if wheel3: self.wheel3.advance()
            
    def lookup(self, letter):
        a = self.wheel3.forward(letter)
        b = self.wheel2.forward(a)
        c = self.wheel1.forward(b)
        d = self.reflector.forward(c)
        e = self.wheel1.backward(d)
        f = self.wheel2.backward(e)
        g = self.wheel3.backward(f)
        return g
    
    def set_positions(self, wheel1_pos, wheel2_pos, wheel3_pos):
        self.wheel1.set_position(wheel1_pos)
        self.wheel2.set_position(wheel2_pos)
        self.wheel3.set_position(wheel3_pos)        

In [4]:
class Bombe(object):
    
    def __init__(self, wheel1_spec, wheel2_spec, wheel3_spec, reflector_spec,
                menu=None, start_signal=None, use_diagonal_board=True, 
                verify_plugboard=True):
        self.connections = []
        self.wheel1_spec = wheel1_spec
        self.wheel2_spec = wheel2_spec
        self.wheel3_spec = wheel3_spec
        self.reflector_spec = reflector_spec
        if menu:
            self.read_menu(menu)
        if start_signal:
            self.test_start = start_signal
        self.use_diagonal_board = use_diagonal_board
        self.verify_plugboard = verify_plugboard
        
    def __getattribute__(self, name):
        if name=='wheel_positions':
            return self.connections[0].scrambler.wheel_positions
        elif name=='wheel_positions_l':
            return self.connections[0].scrambler.wheel_positions_l
        else:
            return object.__getattribute__(self, name)
        
    def __call__(self, start_positions):
        return start_positions, self.test(initial_signal=self.test_start,
            start_positions=start_positions, 
            use_diagonal_board=self.use_diagonal_board,
            verify_plugboard=self.verify_plugboard)
        
    def add_connection(self, bank_before, bank_after, scrambler):
        self.connections += [Connection([bank_before, bank_after], scrambler)]
        
    def read_menu(self, menu):
        self.connections = []
        for item in menu:
            scrambler = Scrambler(self.wheel1_spec, self.wheel2_spec, self.wheel3_spec,
                                  self.reflector_spec,
                                  wheel3_pos=unpos(item.number - 1))
            self.add_connection(item.before, item.after, scrambler)
        most_common_letter = (collections.Counter(m.before for m in menu) + \
                              collections.Counter(m.after for m in menu)).most_common(1)[0][0]
        self.test_start = Signal(most_common_letter, most_common_letter)
        
    def set_positions(self, wheel1_pos, wheel2_pos, wheel3_pos):
        for i, c in enumerate(self.connections):
            c.scrambler.set_positions(wheel1_pos, wheel2_pos, unpos(pos(wheel3_pos) + i))
    
    def test(self, initial_signal=None, start_positions=None, use_diagonal_board=True,
            verify_plugboard=True):
        self.banks = {label: 
                      dict(zip(string.ascii_lowercase, [False]*len(string.ascii_lowercase)))
                      for label in string.ascii_lowercase}
        if start_positions:
            self.set_positions(*start_positions)
        if not initial_signal:
            initial_signal = self.test_start
        self.pending = [initial_signal]
        self.propagate(use_diagonal_board)
        live_wire_count = len([self.banks[self.test_start.bank][w] 
                    for w in self.banks[self.test_start.bank] 
                    if self.banks[self.test_start.bank][w]])
        if live_wire_count < 26:
            if verify_plugboard:
                possibles = self.possible_plugboards()
                return all(s0.isdisjoint(s1) for s0 in possibles for s1 in possibles if s0 != s1)
            else:
                return True
        else:
            return False
        
    def propagate(self, use_diagonal_board):
        while self.pending:
            current = self.pending[0]
            # print("processing", current)
            self.pending = self.pending[1:]
            if not self.banks[current.bank][current.wire]:
                self.banks[current.bank][current.wire] = True
                if use_diagonal_board:
                    self.pending += [Signal(current.wire, current.bank)]
                for c in self.connections:
                    if current.bank in c.banks:
                        if len(set(c.banks)) == 1:
                            other_bank = c.banks[0]
                        else:
                            other_bank = [b for b in c.banks if b != current.bank][0]
                        other_wire = c.scrambler.lookup(current.wire)
                        # print("  adding", other_bank, other_wire, "because", c.banks)
                        self.pending += [Signal(other_bank, other_wire)]
    
    def run(self, run_start=None, wheel1_pos='a', wheel2_pos='a', wheel3_pos='a', use_diagonal_board=True):
        if not run_start:
            run_start = self.test_start
        self.solutions = []
        self.set_positions(wheel1_pos, wheel2_pos, wheel3_pos)
        for run_index in range(26*26*26):
            if self.test(initial_signal=run_start, use_diagonal_board=use_diagonal_board):
                self.solutions += [self.connections[0].scrambler.wheel_positions_l]
            advance3 = True
            advance2 = False
            advance1 = False
            if (run_index + 1) % 26 == 0: advance2 = True
            if (run_index + 1) % (26*26) == 0: advance1 = True
            for c in self.connections:
                c.scrambler.advance(advance1, advance2, advance3)
        return self.solutions
    
    def possible_plugboards(self):
        possibles = set()
        for b in self.banks:
            active = [w for w in self.banks[b] if self.banks[b][w]]
            inactive = [w for w in self.banks[b] if not self.banks[b][w]]
            if len(active) == 1:
                possibles = possibles.union({frozenset((b, active[0]))})
            if len(inactive) == 1:
                possibles = possibles.union({frozenset((b, inactive[0]))})
        return possibles


In [5]:
bombe = Bombe(wheel_i_spec, wheel_ii_spec, wheel_iii_spec, reflector_b_spec)
# len(bombe.banks), bombe.banks['a'] == bombe.banks['b']

In [6]:
test_enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_ii_spec, wheel_ii_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                1, 1, 1,
                '')

In [7]:
test_enigma.set_wheels('a', 'a', 'a')
pt = 'thisisatestmessage'
ct = test_enigma.encipher(pt)
ct

'opgndxcrwomnlnecjz'

In [8]:
cat(test_enigma.wheel_positions_l)

'aas'

In [9]:
menu = [MenuItem(p, c, i+1) for i, (p, c) in enumerate(zip(pt, ct))]
menu

[MenuIem(before='t', after='o', number=1),
 MenuIem(before='h', after='p', number=2),
 MenuIem(before='i', after='g', number=3),
 MenuIem(before='s', after='n', number=4),
 MenuIem(before='i', after='d', number=5),
 MenuIem(before='s', after='x', number=6),
 MenuIem(before='a', after='c', number=7),
 MenuIem(before='t', after='r', number=8),
 MenuIem(before='e', after='w', number=9),
 MenuIem(before='s', after='o', number=10),
 MenuIem(before='t', after='m', number=11),
 MenuIem(before='m', after='n', number=12),
 MenuIem(before='e', after='l', number=13),
 MenuIem(before='s', after='n', number=14),
 MenuIem(before='s', after='e', number=15),
 MenuIem(before='a', after='c', number=16),
 MenuIem(before='g', after='j', number=17),
 MenuIem(before='e', after='z', number=18)]

In [10]:
def make_menu(plaintext, ciphertext):
    return [MenuItem(p, c, i+1) 
            for i, (p, c) in enumerate(zip(plaintext, ciphertext))]

In [11]:
make_menu(pt, ct)

[MenuIem(before='t', after='o', number=1),
 MenuIem(before='h', after='p', number=2),
 MenuIem(before='i', after='g', number=3),
 MenuIem(before='s', after='n', number=4),
 MenuIem(before='i', after='d', number=5),
 MenuIem(before='s', after='x', number=6),
 MenuIem(before='a', after='c', number=7),
 MenuIem(before='t', after='r', number=8),
 MenuIem(before='e', after='w', number=9),
 MenuIem(before='s', after='o', number=10),
 MenuIem(before='t', after='m', number=11),
 MenuIem(before='m', after='n', number=12),
 MenuIem(before='e', after='l', number=13),
 MenuIem(before='s', after='n', number=14),
 MenuIem(before='s', after='e', number=15),
 MenuIem(before='a', after='c', number=16),
 MenuIem(before='g', after='j', number=17),
 MenuIem(before='e', after='z', number=18)]

In [12]:
(collections.Counter(m.before for m in menu) + collections.Counter(m.after for m in menu)).most_common(1)[0][0]

's'

In [13]:
bombe.read_menu(menu)

In [14]:
len(bombe.connections)

18

In [15]:
for c in bombe.connections:
    print(c.banks, cat(c.scrambler.wheel_positions_l))

['t', 'o'] aaa
['h', 'p'] aab
['i', 'g'] aac
['s', 'n'] aad
['i', 'd'] aae
['s', 'x'] aaf
['a', 'c'] aag
['t', 'r'] aah
['e', 'w'] aai
['s', 'o'] aaj
['t', 'm'] aak
['m', 'n'] aal
['e', 'l'] aam
['s', 'n'] aan
['s', 'e'] aao
['a', 'c'] aap
['g', 'j'] aaq
['e', 'z'] aar


In [16]:
':'.join(cat(sorted(c.banks)) for c in bombe.connections)

'ot:hp:gi:ns:di:sx:ac:rt:ew:os:mt:mn:el:ns:es:ac:gj:ez'

In [17]:
':'.join(cat(c.scrambler.wheel_positions_l) for c in bombe.connections)

'aaa:aab:aac:aad:aae:aaf:aag:aah:aai:aaj:aak:aal:aam:aan:aao:aap:aaq:aar'

In [18]:
bombe.test(Signal('t', 't'))

False

In [19]:
bombe.banks['t']

{'a': True,
 'b': True,
 'c': True,
 'd': True,
 'e': True,
 'f': True,
 'g': True,
 'h': True,
 'i': True,
 'j': True,
 'k': True,
 'l': True,
 'm': True,
 'n': True,
 'o': True,
 'p': True,
 'q': True,
 'r': True,
 's': True,
 't': True,
 'u': True,
 'v': True,
 'w': True,
 'x': True,
 'y': True,
 'z': True}

In [20]:
for b in sorted(bombe.banks):
    print(b, ': ', end='')
    for w in sorted(bombe.banks[b]):
        if bombe.banks[b][w]:
            print(w.upper(), end='')
        else:
            print(w, end='')
    print('')

a : ABCDEFGHIJKLMNOPQRSTUVWXYZ
b : AbCDEfGHIJkLMNOPqRSTuvWXyZ
c : ABCDEFGHIJKLMNOPQRSTUVWXYZ
d : ABCDEFGHIJKLMNOPQRSTUVWXyZ
e : ABCDEFGHIJKLMNOPQRSTUVWXYZ
f : AbCDEfGHIJkLMNOPqRSTuvWXyZ
g : ABCDEFGHIJKLMNOPQRSTuVWXYZ
h : ABCDEFGHIJKLMNOPQRSTUVWXYZ
i : ABCDEFGHIJKLMNOPQRSTUvWXYZ
j : ABCDEFGHIjKLMNOPQRSTUVWXYZ
k : AbCDEfGHIJkLMNOPqRSTuvWXyZ
l : ABCDEFGHIJKLMNOPQRSTUVWXYZ
m : ABCDEFGHIJKLMNOPQRSTUVWXYZ
n : ABCDEFGHIJKLMNOPQRSTUVWXYZ
o : ABCDEFGHIJKLMNOPQRSTUVWXYZ
p : ABCDEFGHIJKLMNOPQRSTUVWXYZ
q : AbCDEfGHIJkLMNOPqRSTuvWXyZ
r : ABCDEFGHIJKLMNOPQRSTUVWXYZ
s : ABCDEFGHIJKLMNOPQRSTUVWXYZ
t : ABCDEFGHIJKLMNOPQRSTUVWXYZ
u : AbCDEfgHIJkLMNOPqRSTuvWXyZ
v : AbCDEfGHiJkLMNOPqRSTuvWXyZ
w : ABCDEFGHIJKLMNOPQRSTUVWXYZ
x : ABCDEFGHIJKLMNOPQRSTUVWXYZ
y : AbCdEfGHIJkLMNOPqRSTuvWXyZ
z : ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [21]:
for b in sorted(bombe.banks):
    print(b, ': ', end='')
    for w in sorted(bombe.banks[b]):
        if bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')

a : abcdefghijklmnopqrstuvwxyz
b : a.cde.ghij.lmnop.rst..wx.z
c : abcdefghijklmnopqrstuvwxyz
d : abcdefghijklmnopqrstuvwx.z
e : abcdefghijklmnopqrstuvwxyz
f : a.cde.ghij.lmnop.rst..wx.z
g : abcdefghijklmnopqrst.vwxyz
h : abcdefghijklmnopqrstuvwxyz
i : abcdefghijklmnopqrstu.wxyz
j : abcdefghi.klmnopqrstuvwxyz
k : a.cde.ghij.lmnop.rst..wx.z
l : abcdefghijklmnopqrstuvwxyz
m : abcdefghijklmnopqrstuvwxyz
n : abcdefghijklmnopqrstuvwxyz
o : abcdefghijklmnopqrstuvwxyz
p : abcdefghijklmnopqrstuvwxyz
q : a.cde.ghij.lmnop.rst..wx.z
r : abcdefghijklmnopqrstuvwxyz
s : abcdefghijklmnopqrstuvwxyz
t : abcdefghijklmnopqrstuvwxyz
u : a.cde..hij.lmnop.rst..wx.z
v : a.cde.gh.j.lmnop.rst..wx.z
w : abcdefghijklmnopqrstuvwxyz
x : abcdefghijklmnopqrstuvwxyz
y : a.c.e.ghij.lmnop.rst..wx.z
z : abcdefghijklmnopqrstuvwxyz


In [22]:
bombe.wheel_positions_l

('a', 'a', 'a')

In [23]:
bombe.set_positions('p', 'q', 'r')
for c in bombe.connections:
    print(c.banks, cat(c.scrambler.wheel_positions_l))

['t', 'o'] pqr
['h', 'p'] pqs
['i', 'g'] pqt
['s', 'n'] pqu
['i', 'd'] pqv
['s', 'x'] pqw
['a', 'c'] pqx
['t', 'r'] pqy
['e', 'w'] pqz
['s', 'o'] pqa
['t', 'm'] pqb
['m', 'n'] pqc
['e', 'l'] pqd
['s', 'n'] pqe
['s', 'e'] pqf
['a', 'c'] pqg
['g', 'j'] pqh
['e', 'z'] pqi


In [24]:
# bombe.run()
# print('x')

In [25]:
bombe.set_positions('a', 'a', 'b')
bombe.test(Signal('s', 'a'))

for b in sorted(bombe.banks):
    print(b, ': ', end='')
    for w in sorted(bombe.banks[b]):
        if bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')

a : .b.defghijklmnopqrstuvwxyz
b : a.cde.ghij.lmnop.rst..wx.z
c : .b.defghijklmnopqrstuvwxyz
d : abc.e.gh.jklmnopqrstuvwxyz
e : abcd.fghijklmnopqrstuvwxyz
f : a.c.e.g.ij.lmno..rst..wx.z
g : abcdef.h..klmnopqrstuvwxyz
h : abcde.g.ij.lmno.qrst..wxyz
i : abc.ef.h.jklmnopqrstuvwxyz
j : abcdef.hi.klmnopqrstuvwx.z
k : a.cde.g.ij.lmno..rst..wx.z
l : abcdefghijk.mnopqrstuvwxyz
m : abcdefghijkl.nopqrstuvwxyz
n : abcdefghijklm.opqrstuvwxyz
o : abcdefghijklmn.pqrstuvwxyz
p : abcde.g.ij.lmno.qrst..wxyz
q : a.cde.ghij.lmnop.rst..wx.z
r : abcdefghijklmnopq.stuvwxyz
s : abcdefghijklmnopqr.tuvwxyz
t : abcdefghijklmnopqrs.uvwxyz
u : a.cde.g.ij.lmno..rst..wx.z
v : a.cde.g.ij.lmno..rst..wx.z
w : abcdefghijklmnopqrstuv.xyz
x : abcdefghijklmnopqrstuvw.yz
y : a.cde.ghi..lmnop.rst..wx.z
z : abcdefghijklmnopqrstuvwxy.


In [26]:
bombe.set_positions('a', 'a', 'b')
bombe.test()

for b in sorted(bombe.banks):
    print(b, ': ', end='')
    for w in sorted(bombe.banks[b]):
        if bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')

a : ..........................
b : ..........................
c : ..........................
d : ..........................
e : ....e.....................
f : ..........................
g : ..........................
h : ..........................
i : ..........................
j : ..........................
k : ..........................
l : ...........l..............
m : ............m.............
n : .............n............
o : ..............o...........
p : ..........................
q : ..........................
r : .................r........
s : ..................s.......
t : ...................t......
u : ..........................
v : ..........................
w : ......................w...
x : .......................x..
y : ..........................
z : .........................z


In [27]:
len([bombe.banks['t'][w] for w in bombe.banks['t'] if bombe.banks['t'][w]])

1

In [28]:
# %%timeit
# results = bombe.run()
# print(len(results), ('a', 'a', 'b') in results)

In [29]:
# %%timeit
# results = bombe.run(use_diagonal_board=False)
# print(len(results), ('a', 'a', 'b') in results)

In [30]:
bombe.wheel_positions_l

('a', 'a', 'b')

In [31]:
bombe.test(Signal('t', 't'), ('p', 'p', 'p'))

False

In [32]:
bombe.wheel_positions_l

('p', 'p', 'p')

In [33]:
for b in sorted(bombe.banks):
    print(b, ': ', end='')
    for w in sorted(bombe.banks[b]):
        if bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')

a : abcdefghijklmnop.rst.vwxyz
b : a.cde.g.ij.lmno..rst..wx.z
c : abcdefghijklmnop.rst.vwxyz
d : abcdefghijklmnopqrstuvwxyz
e : abcdefghijklmnopqrstuvwxyz
f : a.cde.g.ij.lmno..rst..wx.z
g : abcdefghijklmnopqrstuvwxyz
h : a.cde.ghijklmnop.rst.vwxyz
i : abcdefghijklmnopqrstuvwxyz
j : abcdefghijklmnopqrstuvwxyz
k : a.cde.ghij.lmnop.rst..wx.z
l : abcdefghijklmnopqrstuvwxyz
m : abcdefghijklmnopqrstuvwxyz
n : abcdefghijklmnopqrstuvwxyz
o : abcdefghijklmnopqrstuvwxyz
p : a.cde.ghijklmnop.rst.vwxyz
q : ...de.g.ij.lmno..rst..wx.z
r : abcdefghijklmnopqrstuvwxyz
s : abcdefghijklmnopqrstuvwxyz
t : abcdefghijklmnopqrstuvwxyz
u : ...de.g.ij.lmno..rst..wx.z
v : a.cde.ghij.lmnop.rst..wx.z
w : abcdefghijklmnopqrstuvwxyz
x : abcdefghijklmnopqrstuvwxyz
y : a.cde.ghij.lmnop.rst..wx.z
z : abcdefghijklmnopqrstuvwxyz


In [34]:
allwheels = itertools.product(string.ascii_lowercase, repeat=3)
len(list(allwheels))

17576

In [35]:
b = Bombe(wheel_i_spec, wheel_ii_spec, wheel_iii_spec, reflector_b_spec, menu=menu)
b(('a', 'a', 'b'))

(('a', 'a', 'b'), True)

In [36]:
b = Bombe(wheel_i_spec, wheel_ii_spec, wheel_iii_spec, reflector_b_spec, menu=menu)(('a', 'a', 'b'))

In [37]:
allwheels = itertools.product(string.ascii_lowercase, repeat=3)

with multiprocessing.Pool() as pool:
    res = pool.map(Bombe(wheel_i_spec, wheel_ii_spec, wheel_iii_spec, reflector_b_spec, menu=menu),
                  allwheels)
[r[0] for r in res if r[1]]

[('a', 'a', 'b')]

In [38]:
def run_multi_bombe(wheel1_spec, wheel2_spec, wheel3_spec, reflector_spec, menu,
                    start_signal=None, use_diagonal_board=True, 
                    verify_plugboard=True):
    allwheels = itertools.product(string.ascii_lowercase, repeat=3)

    with multiprocessing.Pool() as pool:
        res = pool.map(Bombe(wheel1_spec, wheel2_spec, wheel3_spec, 
            reflector_spec, menu=menu, start_signal=start_signal, 
            use_diagonal_board=use_diagonal_board, 
            verify_plugboard=verify_plugboard),
                  allwheels)
    return [r[0] for r in res if r[1]]

In [39]:
[r[0] for r in res if r[1]]

[('a', 'a', 'b')]

In [40]:
# Setting sheet line 31 from http://www.codesandciphers.org.uk/enigma/enigma3.htm
# Enigma simulation settings are 
# http://enigma.louisedade.co.uk/enigma.html?m3;b;b153;AFTX;AJFE;AU-BG-EY-FP-HL-IN-JZ-OS-QR-TX
w_enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_pegs,
                wheel_v_spec, wheel_v_pegs,
                wheel_iii_spec, wheel_iii_pegs,
                6, 20, 24,
                'ua pf rq so ni ey bg hl tx zj')

In [41]:
w_enigma.set_wheels('j', 'e', 'b')
tuple(unpos(p) for p in w_enigma.wheel_positions)

('e', 'l', 'e')

In [42]:
w_enigma.set_wheels('j', 'e', 'b')
pt = 'someplaintext'
ct = w_enigma.encipher(pt)
ct

'dhnpforeeimgg'

In [43]:
w_enigma.wheel_positions_l

('j', 'e', 'o')

In [44]:
w_menu = [MenuItem(p, c, i+1) for i, (p, c) in enumerate(zip(pt, ct))]
w_menu

[MenuIem(before='s', after='d', number=1),
 MenuIem(before='o', after='h', number=2),
 MenuIem(before='m', after='n', number=3),
 MenuIem(before='e', after='p', number=4),
 MenuIem(before='p', after='f', number=5),
 MenuIem(before='l', after='o', number=6),
 MenuIem(before='a', after='r', number=7),
 MenuIem(before='i', after='e', number=8),
 MenuIem(before='n', after='e', number=9),
 MenuIem(before='t', after='i', number=10),
 MenuIem(before='e', after='m', number=11),
 MenuIem(before='x', after='g', number=12),
 MenuIem(before='t', after='g', number=13)]

In [45]:
allwheels = itertools.product(string.ascii_lowercase, repeat=3)

with multiprocessing.Pool() as pool:
    res = pool.map(Bombe(wheel_i_spec, wheel_v_spec, wheel_iii_spec, reflector_b_spec, 
                         menu=w_menu, verify_plugboard=False),
                  allwheels)
[r[0] for r in res if r[1]]

[('a', 'y', 'm'),
 ('c', 'p', 'v'),
 ('c', 's', 'f'),
 ('c', 'w', 'j'),
 ('d', 'r', 'k'),
 ('e', 'l', 'f'),
 ('e', 's', 'v'),
 ('e', 'y', 'd'),
 ('e', 'y', 'o'),
 ('f', 'z', 'x'),
 ('g', 'b', 'l'),
 ('g', 'c', 'd'),
 ('g', 'c', 'f'),
 ('g', 'j', 'p'),
 ('h', 'c', 'i'),
 ('h', 'm', 'w'),
 ('h', 'o', 'd'),
 ('h', 'p', 'b'),
 ('h', 's', 't'),
 ('i', 'b', 's'),
 ('i', 'v', 'b'),
 ('j', 'y', 'u'),
 ('k', 'b', 'x'),
 ('k', 'f', 't'),
 ('k', 'l', 'e'),
 ('k', 'l', 'm'),
 ('k', 'r', 'z'),
 ('k', 's', 'p'),
 ('l', 'd', 'z'),
 ('l', 'i', 'y'),
 ('l', 'y', 'f'),
 ('m', 'b', 'h'),
 ('m', 'p', 'l'),
 ('n', 'l', 'r'),
 ('o', 'k', 'x'),
 ('p', 'a', 'g'),
 ('p', 'c', 'v'),
 ('p', 'f', 'o'),
 ('p', 'm', 'i'),
 ('p', 'x', 'n'),
 ('p', 'x', 'p'),
 ('q', 'q', 'n'),
 ('q', 'r', 'w'),
 ('q', 'v', 'l'),
 ('q', 'x', 't'),
 ('s', 'a', 'h'),
 ('s', 'h', 'v'),
 ('s', 'l', 'p'),
 ('s', 'l', 's'),
 ('u', 'r', 'h'),
 ('v', 'v', 'v'),
 ('v', 'x', 'a'),
 ('w', 'j', 'z'),
 ('w', 'k', 'u'),
 ('x', 'f', 'p'),
 ('x', 'j'

In [46]:
len([r[0] for r in res if r[1]])

62

In [47]:
allwheels = itertools.product(string.ascii_lowercase, repeat=3)

with multiprocessing.Pool() as pool:
    res = pool.map(Bombe(wheel_i_spec, wheel_v_spec, wheel_iii_spec, reflector_b_spec, 
                         menu=w_menu, verify_plugboard=True),
                  allwheels)
[r[0] for r in res if r[1]]

[('c', 'p', 'v'),
 ('c', 's', 'f'),
 ('e', 'l', 'f'),
 ('g', 'c', 'f'),
 ('j', 'y', 'u'),
 ('o', 'k', 'x'),
 ('p', 'a', 'g'),
 ('q', 'q', 'n'),
 ('q', 'v', 'l'),
 ('q', 'x', 't'),
 ('s', 'l', 'p'),
 ('u', 'r', 'h'),
 ('y', 'n', 'c')]

In [48]:
run_multi_bombe(wheel_i_spec, wheel_v_spec, wheel_iii_spec, reflector_b_spec, w_menu)

[('c', 'p', 'v'),
 ('c', 's', 'f'),
 ('e', 'l', 'f'),
 ('g', 'c', 'f'),
 ('j', 'y', 'u'),
 ('o', 'k', 'x'),
 ('p', 'a', 'g'),
 ('q', 'q', 'n'),
 ('q', 'v', 'l'),
 ('q', 'x', 't'),
 ('s', 'l', 'p'),
 ('u', 'r', 'h'),
 ('y', 'n', 'c')]

In [49]:
# allwheels = itertools.product(string.ascii_lowercase, repeat=3)

# with multiprocessing.Pool() as pool:
#     res = pool.map(Bombe(wheel_i_spec, wheel_v_spec, wheel_iii_spec, reflector_b_spec, 
#                          menu=w_menu, start_signal=Signal('t', 'x')),
#                   allwheels)
# [r[0] for r in res if r[1]]

In [50]:
len([r[0] for r in res if r[1]])

13

In [51]:
w_bombe = Bombe(wheel_i_spec, wheel_v_spec, wheel_iii_spec, reflector_b_spec, 
                         menu=w_menu)

In [52]:
w_bombe.test_start

Signal(bank='e', wire='e')

In [53]:
w_bombe.test(start_positions=('e', 'l', 'f'))

True

In [54]:
w_bombe.test(Signal('t', 'x'), ('e', 'l', 'f'))

True

In [55]:
r = w_bombe.test(start_positions=('c', 'p', 'v'))
print(r)
for b in sorted(w_bombe.banks):
    print(b, ': ', end='')
    for w in sorted(w_bombe.banks[b]):
        if w_bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')

True
a : a.cdefghi..l.nopqrst.vwx..
b : ....efghi...mnop.......x..
c : a....fg.i..lmn.p.r.t...x..
d : a...efghij.lmn...rstu..xyz
e : ab.defghijklmnopqrstuvwxyz
f : abcde.ghijklmnopqrstuvwxyz
g : abcdef.hijklmnopqrstuvwxyz
h : ab.defghi...mnopq.stuvwx..
i : abcdefgh.jklmnopqrstuvwxyz
j : ...defg.i...mn.p...t...x..
k : ....efg.i..lmnop.r.t...x..
l : a.cdefg.i.k.mnopqr.t..wxyz
m : .bcdefghijklmnopqrstuvwxyz
n : abcdefghijklmnopqr.tuvwxyz
o : ab..efghi.klmnopqr.t...xyz
p : abc.efghijklmnopqrstuvwxyz
q : a...efghi..lmnop.r.t...x..
r : a.cdefg.i.klmnopqrst..w..z
s : a..defghi...m..p.rstuv.xyz
t : a.cdefghijklmnopqrstuvwxyz
u : ...defghi...mn.p..st...x..
v : a...efghi...mn.p..st...x..
w : a...efghi..lmn.p.r.t...x..
x : abcdefghijklmnopq.stuvwxyz
y : ...defg.i..lmnop..st...x..
z : ...defg.i..lmnop.rst...x..


In [56]:
r = w_bombe.test(start_positions=('e', 'l', 'f'))
print(r)
for b in sorted(w_bombe.banks):
    print(b, ': ', end='')
    for w in sorted(w_bombe.banks[b]):
        if w_bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')

True
a : abcdefghi..lmnop.rst.v.x.z
b : a...ef.hi..lmnop.r.t...x..
c : a..defg.i..lmn.p.rst...x..
d : a.c.efghi.klmnopqrstu.wxyz
e : abcdefghijklmnopqrstuvwx.z
f : abcdefghijklmno.qrstuvwxyz
g : a.cdefghijklmnopqrstuvwxyz
h : ab.defghi...mnopqrstuvwxyz
i : abcdefghijklm.opqrstuvwxyz
j : ....efg.i..lmnop...t...x..
k : ...defg.i..lmn.p..st...x..
l : abcdefg.ijklmnopqrstuv.x..
m : abcdefghijkl.nopqrstuvwxyz
n : abcdefgh.jklmnopqrstuvwxyz
o : ab.defghij.lmnop.r.tuvwxyz
p : abcde.ghijklmnopqrstuvwxyz
q : ...defghi..lmn.p..st...x..
r : abcdefghi..lmnop.rst.v.x.z
s : a.cdefghi.klmn.pqr.tuvwxyz
t : abcdefghijklmnopqrstuvw.yz
u : ...defghi..lmnop..st...x..
v : a...efghi..lmnop.rst...x..
w : ...defghi...mnop..st...x..
x : abcdefghijklmnopqrs.uvwxyz
y : ...d.fghi...mnop..st...x..
z : a..defghi...mnop.rst...x..


In [57]:
ps = w_bombe.possible_plugboards()
ps

{frozenset({'t', 'x'}),
 frozenset({'i', 'n'}),
 frozenset({'m'}),
 frozenset({'e', 'y'}),
 frozenset({'f', 'p'}),
 frozenset({'b', 'g'})}

In [58]:
all(s0.isdisjoint(s1) for s0 in ps for s1 in ps if s0 != s1)

True

In [59]:
s = set()
f1 = frozenset((1, 2))
f2 = frozenset((3, 4))
f3 = frozenset((2, 3))
s = s.union({f1})
s = s.union({f2})
s = s.union({f1})
s = s.union({f3})
s, f1, f2, f3

({frozenset({1, 2}), frozenset({2, 3}), frozenset({3, 4})},
 frozenset({1, 2}),
 frozenset({3, 4}),
 frozenset({2, 3}))

In [60]:
all(s0.isdisjoint(s1) for s0 in s for s1 in s if s0 != s1)

False

In [61]:
{1, 2}.isdisjoint({1, 6})

False

# Tsest

In [62]:
target_ct = ''.join(c.lower() for c in 'SLGNC SZXLT KZEBG HSTGY WDMPR' if c in string.ascii_letters)
target_ct

'slgncszxltkzebghstgywdmpr'

In [63]:
target_pt = ''.join(c.lower() for c in 'Theyw erede tecte d byBri tishs hipsi nclud' if c in string.ascii_letters)
target_pt

'theyweredetectedbybritishshipsinclud'

In [64]:
tbt_menu = [MenuItem(p, c, i+1) for i, (p, c) in enumerate(zip(target_pt[4:24], target_ct[4:24]))]
tbt_menu

[MenuIem(before='w', after='c', number=1),
 MenuIem(before='e', after='s', number=2),
 MenuIem(before='r', after='z', number=3),
 MenuIem(before='e', after='x', number=4),
 MenuIem(before='d', after='l', number=5),
 MenuIem(before='e', after='t', number=6),
 MenuIem(before='t', after='k', number=7),
 MenuIem(before='e', after='z', number=8),
 MenuIem(before='c', after='e', number=9),
 MenuIem(before='t', after='b', number=10),
 MenuIem(before='e', after='g', number=11),
 MenuIem(before='d', after='h', number=12),
 MenuIem(before='b', after='s', number=13),
 MenuIem(before='y', after='t', number=14),
 MenuIem(before='b', after='g', number=15),
 MenuIem(before='r', after='y', number=16),
 MenuIem(before='i', after='w', number=17),
 MenuIem(before='t', after='d', number=18),
 MenuIem(before='i', after='m', number=19),
 MenuIem(before='s', after='p', number=20)]

In [65]:
tbt_bombe = Bombe(wheel_iii_spec, wheel_i_spec, wheel_ii_spec, reflector_b_spec, 
                         menu=tbt_menu)

In [66]:
tbt_wheel_posns = run_multi_bombe(wheel_iii_spec, wheel_i_spec, wheel_ii_spec, reflector_b_spec, tbt_menu)
tbt_wheel_posns

[('k', 'r', 'n')]

In [67]:
tbt_bombe.test_start

Signal(bank='e', wire='e')

In [68]:
r = tbt_bombe.test(start_positions=('l', 's', 'd'))
print(r)
for b in sorted(w_bombe.banks):
    print(b, ': ', end='')
    for w in sorted(w_bombe.banks[b]):
        if w_bombe.banks[b][w]:
            print(w, end='')
        else:
            print('.', end='')
    print('')


False
a : abcdefghi..lmnop.rst.v.x.z
b : a...ef.hi..lmnop.r.t...x..
c : a..defg.i..lmn.p.rst...x..
d : a.c.efghi.klmnopqrstu.wxyz
e : abcdefghijklmnopqrstuvwx.z
f : abcdefghijklmno.qrstuvwxyz
g : a.cdefghijklmnopqrstuvwxyz
h : ab.defghi...mnopqrstuvwxyz
i : abcdefghijklm.opqrstuvwxyz
j : ....efg.i..lmnop...t...x..
k : ...defg.i..lmn.p..st...x..
l : abcdefg.ijklmnopqrstuv.x..
m : abcdefghijkl.nopqrstuvwxyz
n : abcdefgh.jklmnopqrstuvwxyz
o : ab.defghij.lmnop.r.tuvwxyz
p : abcde.ghijklmnopqrstuvwxyz
q : ...defghi..lmn.p..st...x..
r : abcdefghi..lmnop.rst.v.x.z
s : a.cdefghi.klmn.pqr.tuvwxyz
t : abcdefghijklmnopqrstuvw.yz
u : ...defghi..lmnop..st...x..
v : a...efghi..lmnop.rst...x..
w : ...defghi...mnop..st...x..
x : abcdefghijklmnopqrs.uvwxyz
y : ...d.fghi...mnop..st...x..
z : a..defghi...mnop.rst...x..


In [69]:
ps = tbt_bombe.possible_plugboards()
ps

set()